In [1]:
import matplotlib.image as mpimg 
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


training_images = np.zeros((87,576,432,3))
test_set_images = np.zeros((23,576,432,3))
train_set_labels = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
test_set_labels = np.array([[0,0,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0]]) 


tempstr = ""
for i in range(0,87):
    tempstr = "Training/" + str(i+1) + ".jpeg"
    image = mpimg.imread(tempstr)
    training_images[i] = image
    
for i in range(1, 24):
    tempstr = "Test/" + str(i+1) + ".jpeg"
    image = mpimg.imread(tempstr)
    test_set_images[i - 1] = image

train_set_flatten = training_images.reshape(training_images.shape[0], -1).T
test_set_flatten = test_set_images.reshape(test_set_images.shape[0], -1).T
train_set = train_set_flatten/255

print("done")
test_set = test_set_flatten/255


done


In [76]:
layer_dims = [20, 10, 5, 3, 1]

def initialize_parameters(layer_dims, X):
    #layer_dims is an array that contains the number of neurons in each layer
    #X is the training set. We need to use it to find the dimensions of W1
    parameters = {}
    parameters["W1"] = np.random.randn(layer_dims[0], X.shape[0]) * 0.01
    #print("W1 shape " + str(parameters["W1"].shape))
    parameters["b1"] = np.random.randn(layer_dims[0], 1)  * 0.01
    #print("b1 shape " + str(parameters["b1"].shape))
    for l in range(1 , len(layer_dims)):
        parameters["W" + str(l + 1)] = np.random.randn(layer_dims[l], layer_dims[l - 1])* 0.01
        #print("W" + str(l  +  1) + " shape " + str(parameters["W" + str(l + 1)].shape))
        parameters["b" + str(l + 1)] = np.random.randn(layer_dims[l], 1) * 0.01
        #print("b" + str(l  +  1) + " shape " + str(parameters["b" + str(l + 1)].shape))
    return parameters

def sigmoid(z):
    
    return 1/(1 + np.exp(-1*z))

def activate(z, function):
    if function == "sigmoid":
        return sigmoid(z)
        
    if function == "tanh":
        return np.tanh(z)

def forward_prop(parameters, layer_dims, X):
    #calculates all Z in the method, then call a helper method to 
    #find A depending on the activation
    #returns a vector with predicted values
    #finding A requires knowing the activation function
    #you must cache all Z values for GD in backprop
    Z_cache = {}
    A_cache = {}
    layers = len(layer_dims) 
    A = X
    A_cache["A0"] = X
    for i in range(1, layers ):
        #i's last value is len(layer_dims) - 2
        Z_cache["Z" + str(i)] = np.dot(parameters["W" + str(i)], A) + parameters["b" + str(i)]
        A = activate(Z_cache["Z" + str(i)], "tanh")
        A_cache["A" + str(i)] = A
        
    Z_cache["Z" + str(layers)] = np.dot(parameters["W" + str(layers)], A_cache["A" + str(len(layer_dims) - 1)]) + parameters["b" + str(layers)]
    A_cache["A" + str(layers)] = activate(Z_cache["Z" + str(layers)], "sigmoid")
    
    return Z_cache, A_cache


def back_prop(Y, X, parameters, Z_cache, A_cache,  layer_dims):
    #find all gradients dW, db, dA, dZ
    #update all W, B parameters
    grads = {}
    layers = len(layer_dims)
    m = X.shape[1]
    grads["dA" + str(layers)] = - np.divide(Y, A_cache["A" + str(layers)]) + np.divide(1 - Y, 1 -  A_cache["A" + str(layers)]) 
    grads["dZ" + str(layers)] = A_cache["A" + str(layers)] - Y
    for l in range(layers, 0, -1):
        grads["db" + str(l)] = 1/m * np.sum(grads["dZ" + str(l)], axis = 1, keepdims = True)
        grads["dA" + str(l - 1)] = np.dot(parameters["W" + str(l)].T, grads["dZ" + str(l)])
        grads["dW" + str(l)] = 1/m * np.dot(grads["dZ" + str(l)], A_cache["A" + str(l - 1)].T)
        
        if l > 1:
            
            grads["dZ" + str(l - 1)] =  grads["dA" + str(l-1)] * (1 - np.power(A_cache["A" + str(l-1)], 2))
        
    return grads

def optimize(X, Y, Z_cache, A_cache, parameters, its, learning_rate, grads, layer_dims):
    layers = len(layer_dims)
    x = [1, 10, 25, 50, 100, 250, 500, 1000, 1750]
    for i in range(its):
        grads = back_prop(Y , X, parameters, Z_cache, A_cache, layer_dims)
        for l in range(1, layers + 1):
            parameters["W" + str(l)] = parameters["W" + str(l)]  - learning_rate*grads["dW" + str(l)]
                
            parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate*grads["db" + str(l)]
        Z_cache, A_cache = forward_prop(parameters, layer_dims, X)
        
        if(i in x):
            print("Completed " + str(i) + " iterations")
        
    return parameters

def predict(X, parameters, layer_dims):
    layers = len(layer_dims)
    for i in range(1, layers):
        if(i == 1):
            Z = np.dot(parameters["W1"], X) + parameters["b1"]
            
        else:
            Z = np.dot(parameters["W" + str(i)], A) + parameters["b" + str(i)]
                
        print("Z" + str(i) + " ", Z)
        A = np.tanh(Z)
        
    
    
    Z = np.dot(parameters["W" + str(layers)], A) + parameters["b" + str(layers)]
    print("W" , parameters["W" + str(layers)])
    print("W shape",  str(parameters["W" + str(layers)].shape))
    
    print("final Z " , Z)
    
    A = sigmoid(Z)
    print("final A " , A)
    predict = np.array(A, copy = True)
    predict[A < 0.75] = 0
    predict[A >= 0.75] = 1
    predict = np.array(predict)
    print("predict array ", predict)
    return predict
            
        
def L_layer_deeplearning_model(layer_dims, Train, Train_labels, Test, Test_labels, its, learning_rate):
    parameters = initialize_parameters(layer_dims, Train)
    Z_cache, A_cache = forward_prop(parameters, layer_dims, Train)
    
    #print("Z1 shape " + str(Z_cache["Z1"].shape))
    #print("Z2 shape " + str(Z_cache["Z2"].shape))
    #print("A1 shape " + str(A_cache["A1"].shape))
    #print("A2 shape " + str(A_cache["A2"].shape))
    
    grads = back_prop(Train_labels, Train, parameters, Z_cache, A_cache, layer_dims)
    new_para = optimize(Train, Train_labels, Z_cache, A_cache, parameters, its, learning_rate, grads, layer_dims)
    prediction = predict(Test, new_para, layer_dims)
    
    print("test accuracy: {} %".format(100 - np.mean(np.abs(prediction - Test_labels)) * 100))
       

In [ ]:
import time

layer_dims = [3, 5, 3, 1]  
start = time.time()
L_layer_deeplearning_model(layer_dims, train_set, train_set_labels, test_set, test_set_labels, 1500, 0.01 )
end = time.time()
print("time to run: "  + str(end-start))
